# Robot Range Calculation - 3D Analysis

This notebook demonstrates advanced range calculation and visualization for a robot's working space using three joint axes.

## Overview
This example showcases:
1. **3D Robot Range Analysis**: Computing collision-free configurations across a 3-dimensional joint parameter space
2. **3D Range Visualization**: Interactive 3D plotting of valid vs. invalid joint combinations
3. **Multi-Joint Analysis**: Analyzing complex interactions between multiple joint parameters

## Key Components
- `RangeCalculator`: Computes collision-free working space for specified robot joints
- `MoveManager`: Controls robot joint movements for visualization
- `RobotVisualizer` & `MoveVisualizer`: Display robot configurations

## Analysis Setup
- **Robot**: Simple 3-link robot with revolute and prismatic joints
- **Target Links**: Hand and finger links (parts we want to keep collision-free)
- **Obstacle Links**: Obstacle link (environment constraint)
- **Axes**: Three joint parameters (revolute_joint, prismatic_joint, prismatic_joint2)

## Expected Results
The 3D range plot will show valid operating volumes in the joint parameter space, providing comprehensive collision analysis for complex robot motions.

Let's start by setting up logging and importing libraries:

In [ ]:
import logging

import numpy as np

from linkmotion.visual import RobotVisualizer, MoveVisualizer
from linkmotion import (
    Robot,
    Link,
    Joint,
    JointType,
    Transform,
    MoveManager,
)
from linkmotion.range.range import RangeCalculator

logging.basicConfig(level=logging.DEBUG)

## Robot Definition Function

Creating a simple test robot with multiple joint types and an obstacle:

In [ ]:
def simple_robot():
    """Creates a simple robot with two joints for testing."""
    robot = Robot()

    base_link = Link.from_sphere(
        name="base_link", radius=0.1, center=np.array([0, 0, 0])
    )
    arm_link = Link.from_cylinder(
        name="arm_link",
        radius=0.1,
        height=1.0,
        default_transform=Transform(translate=np.array([0, 0, 0.5])),
    )
    hand_link = Link.from_box(
        name="hand_link",
        extents=np.array([0.1, 0.1, 0.1]),
        default_transform=Transform(translate=np.array([0, 0, 1.0])),
        color=np.array([0, 1, 0, 1]),
    )
    finger_link = Link.from_box(
        name="finger_link",
        extents=np.array([0.05, 0.05, 0.1]),
        default_transform=Transform(translate=np.array([0, 0, 1.1])),
        color=np.array([1, 0, 0, 1]),
    )
    obstacle_link = Link.from_box(
        name="obstacle_link",
        extents=np.array([10, 10, 0.1]),
        default_transform=Transform(translate=np.array([0, 0, 1.5])),
    )

    revolute_joint = Joint(
        name="revolute_joint",
        type=JointType.REVOLUTE,
        parent_link_name="base_link",
        child_link_name="arm_link",
        direction=np.array([1, 0, 0]),
        center=np.array([0, 0, 0.0]),
        min_=-np.pi / 2,
        max_=np.pi / 2,
    )

    prismatic_joint = Joint(
        name="prismatic_joint",
        type=JointType.PRISMATIC,
        parent_link_name="arm_link",
        child_link_name="hand_link",
        direction=np.array([0, 1, 0]),
        center=np.array([0, 0, 1.0]),
        min_=-10,
        max_=10,
    )

    prismatic_joint2 = Joint(
        name="prismatic_joint2",
        type=JointType.PRISMATIC,
        parent_link_name="hand_link",
        child_link_name="finger_link",
        direction=np.array([0, 0, 1]),
        center=np.array([0, 0, 1.1]),
        min_=-10,
        max_=10,
    )

    robot.add_link(base_link)
    robot.add_link(arm_link)
    robot.add_link(hand_link)
    robot.add_link(finger_link)
    robot.add_link(obstacle_link)
    robot.add_joint(revolute_joint)
    robot.add_joint(prismatic_joint)
    robot.add_joint(prismatic_joint2)

    return robot


robot = simple_robot()

## Default Robot Visualization

Displaying the robot in its default configuration:

In [ ]:
RobotVisualizer.robot(robot)

## Output Example
![OutputExample](./img/range_2axes/range_2axes1.png)

## Sample Robot Configuration

Demonstrating joint movement and visualization:

In [ ]:
mm = MoveManager(robot)
mm.move("revolute_joint", np.pi / 4)
mm.move("prismatic_joint", 0.5)
MoveVisualizer.robot(mm)

## Output Example
![OutputExample](./img/range_2axes/range_2axes2.png)

## Range Calculation - 3D Analysis

Computing the collision-free working space for three joint parameters (higher resolution analysis):

In [ ]:
calculator = RangeCalculator(
    simple_robot(), {"hand_link", "finger_link"}, {"obstacle_link"}
)
calculator.add_axis("revolute_joint", np.linspace(-np.pi / 2, np.pi / 2, 40))
calculator.add_axis("prismatic_joint", np.linspace(-3, 3, 60))
calculator.add_axis("prismatic_joint2", np.linspace(-1, 1, 50))
calculator.execute()

## 3D Range Visualization Plot

Displaying the interactive 3D range plot showing collision-free volumes in joint parameter space:

In [ ]:
calculator.plot()

## Output Example
![OutputExample](./img/range_3axes/range_3axes1.png)

## Specific 3D Configuration Test

Testing a specific point from the 3D range plot to verify collision-free operation with all three joints:

In [ ]:
mm = MoveManager(robot)
mm.move("revolute_joint", 0.36)
mm.move("prismatic_joint", 0.76)
mm.move("prismatic_joint2", 0.3)
MoveVisualizer.robot(mm)

## Output Example
![OutputExample](./img/range_2axes/range_2axes4.png)